## This notebook handles the pre-processing needed to modify thereaction bounds in a cobra model to facilitate running Eflux2

In [1]:
import sys
sys.path.append('../src')
sys.path.append('/Users/mahs128/Repos/CONCERTO')
from concerto.helpers.load_model_from_git import load_model_from_git
from eflux2 import EFlux2
import cobra
import pandas as pd
import numpy as np
import gurobipy
from fba_utils import convert_transcriptomics_to_enzyme_activity

### 1. Create surrogate model for the reference strain by tightening bounds on fluxes

#### 1a. Obtain optimized fluxes from the model constrained to the same criteria used to select the reference strain

#### 1b. Set optimized fluxes of reactions of interest in the model upper bounds 

#### 1c. Run flux variability analysis to obtain (reasonably) tight bounds on the fluxes for all other reactions in the model

In [2]:
# Load SBML model
# syn_model = cobra.io.read_sbml_model('../models/iJB785_w_sucrose_transport.xml')
syn_model = cobra.io.read_sbml_model('../models/syn_elong.xml')

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"


In [3]:
print(syn_model.objective)

Maximize
1.0*BIOMASS__1 - 1.0*BIOMASS__1_reverse_063c7


In [4]:
# Run optimization on cobra model to get fluxes for reactions of interest
# sucrose production ('EX_sucr_e') and CO2 uptake ('EX_co2_e') and biomass ('BIOMASS__1')
opt_df = syn_model.optimize().to_frame()
display(opt_df)
display(opt_df.loc['EX_sucr_e'])
display(opt_df.loc['EX_co2_e'])
display(opt_df.loc['BIOMASS__1'])

,fluxes,reduced_costs
EX_gln__L_e,0.000000,0.000000e+00
EX_hco3_e,-1.990000,0.000000e+00
EX_mn2_e,0.000000,0.000000e+00
EX_arg__L_e,0.000000,0.000000e+00
ADPT,0.000006,8.881784e-16
...,...,...
ZDS,0.000323,0.000000e+00
RDXRr,0.000000,-0.000000e+00
MPTSS,0.000000,0.000000e+00
NOR,0.000323,0.000000e+00


fluxes           0.011
reduced_costs    0.000
Name: EX_sucr_e, dtype: float64

fluxes          -0.263564
reduced_costs    0.000000
Name: EX_co2_e, dtype: float64

fluxes           5.390524e-02
reduced_costs    5.684342e-14
Name: BIOMASS__1, dtype: float64

In [5]:
# Specify upper/lower bounds of model for excluded reactions
for rxn in ['EX_sucr_e', 'EX_co2_e', 'BIOMASS__1']:
   syn_model.reactions.get_by_id(rxn).lower_bound = opt_df.loc[rxn, 'fluxes']

   print(rxn, ": ", syn_model.reactions.get_by_id(rxn).lower_bound, ";  ", syn_model.reactions.get_by_id(rxn).upper_bound)

EX_sucr_e :  0.011 ;   1000.0
EX_co2_e :  -0.263564375437085 ;   1000.0
BIOMASS__1 :  0.053905238809111765 ;   2.0


In [6]:
# Create reactions list exclude reactions of interest: sucrose production ('EX_sucr_e') and CO2 uptake ('EX_co2_e') and biomass ('BIOMASS__1') 
rxn_list = [r.id for r in syn_model.reactions if (r.id not in ['EX_sucr_e', 'EX_co2_e', 'BIOMASS__1'])]

In [7]:
# Run FVA (~30-40 seconds)
FVA_object = cobra.flux_analysis.flux_variability_analysis(model=syn_model, reaction_list=rxn_list, 
                                                           fraction_of_optimum=0.85, processes=8)

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/k9/b8pxky2572sdtgy2vnxhcljw0000gn/T/tmpulf48yrc.lp
Reading time = 0.00 seconds
: 928 rows, 2181 columns, 9211 nonzeros
Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/k9/b8pxky2572sdtgy2vnxhcljw0000gn/T/tmpkpquzsrs.lp
Reading time = 0.00 seconds
: 928 rows, 2181 columns, 9211 nonzeros
Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/k9/b8pxky2572sdtgy2vnxhcljw0000gn/T/tmpk4g90c7e.lp
Reading time = 0.00 seconds
: 928 rows, 2181 columns, 9211 nonzeros
Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/k9/b8pxky2572sdtgy2vnxhcljw0000gn/T/tmpuabcugjy.lp
Reading time = 0.00 seconds
: 928 rows, 2181 columns, 9211 nonzeros
Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/k9/b8pxky2572s

In [8]:
FVA_object

,minimum,maximum
EX_gln__L_e,0.000000,0.000000
EX_hco3_e,-1.990000,-1.989450
EX_mn2_e,0.000000,0.000000
EX_arg__L_e,0.000000,0.000000
ADPT,0.000006,0.000006
...,...,...
ZDS,0.000000,0.000339
RDXRr,0.000000,0.000000
MPTSS,0.000000,0.000000
NOR,0.000000,0.000339


In [9]:
zero_flux_from_FVA = [i for i in FVA_object.index[(FVA_object.minimum==0) & (FVA_object.maximum==0)]]
print("# of 0-flux reactions from FVA:", len(zero_flux_from_FVA))
print("Percent of fluxes in the model with of 0-values from FVA:", round(100*len(zero_flux_from_FVA)/len(opt_df),2),"%")

# of 0-flux reactions from FVA: 252
Percent of fluxes in the model with of 0-values from FVA: 23.12 %


 ### 2. Next, convert the transcriptomics data to enzyme data (Take the min of subunits, take the sum of isozymes)

In [10]:
# Load transciptomics data
transcriptomics_fname = "processed_data/cleaned_transcriptomics.csv"
transcriptomics_df = pd.read_csv(transcriptomics_fname, index_col="Label")
transcriptomics_df.head()

,Se_axen_d4_1,Se_axen_d4_2,Se_axen_d4_3,Se_axen_d6_1,Se_axen_d6_2,Se_axen_d6_3,Se_axen_d8_1,Se_axen_d8_2,Se_axen_d8_3
Label,,,,,,,,,
SYNPCC7942_RS00005,16290,17761,15101,14567,13967,12653,7016,8855,8689
SYNPCC7942_RS00010,7185,7502,6565,10086,7546,7705,3747,6670,6960
SYNPCC7942_RS00015,24176,26304,22781,23926,21306,20412,13440,17253,17053
SYNPCC7942_RS00020,35119,35145,25895,37701,34175,30569,24835,39769,30280
SYNPCC7942_RS00025,6891,7803,6607,6347,6284,6844,3326,4146,3180


In [11]:
# Set reference strain
ref_rep = transcriptomics_df.columns[8] # 'Se_axen_d8_3'
ref_rep

'Se_axen_d8_3'

# Function to create dictionary of reactions to isozyme sets (corresponding genes from gene reaction rules)
def get_gpr_dict(model):
    """ Returns the gene reaction rule (GPR) for each reaction in the model."""
    # Parse GPR into a dict containing isozymes (separated by 'or')
    # Each isozyme has a set of subunits (separated by 'and')
    gpr_dict = dict()
    for r in model.reactions:
        if r.gene_reaction_rule:
            isozymes = set()
            for isozyme in [isozyme.strip('() ') for isozyme in r.gene_reaction_rule.split(' or ')]:
                isozymes.add(frozenset(gene.strip('() ') for gene in isozyme.split(' and ')))
            gpr_dict[r] = isozymes
    
    return gpr_dict



gpr_dict = get_gpr_dict(syn_model)
display(gpr_dict)

# Check which genes (listed in observed transcriptomics data) are missing from the cobra model
# list_of_gprs_in_model = [str(syn_model.reactions.get_by_id(r.id).gpr).split(" and ") for r in syn_model.reactions]
list_of_gprs_in_model = [g.id for g in syn_model.genes]

genes_not_in_model = [g for g in transcriptomics_df.index if g not in list_of_gprs_in_model]
genes_not_in_data = [g for g in list_of_gprs_in_model if g not in transcriptomics_df.index]
genes_in_model_and_data = [g for g in list_of_gprs_in_model if g in transcriptomics_df.index]


print('# of genes in model:', len(list_of_gprs_in_model))
print('# of genes in data:', len(transcriptomics_df.index))
print('# of genes_not_in_model:', len(genes_not_in_model))
print('# of genes_not_in_data:', len(genes_not_in_data))
print('# of genes_in_model_and_data:', len(genes_in_model_and_data))

print('# of reactions in model:', len([r.id for r in syn_model.reactions]))

genes_not_in_data

# Code shared by Jeremy in Teams chat...
from cobra import Reaction, Gene

def gene_expression_to_enzyme_activity(gpr: dict[Reaction, list[list[Gene]]], expression: dict[Gene, float]):
    """Map gene expression to enzyme activity
    inputs:
        gpr: dictionary of reactions (keys) to list of list of genes (values) for the correpsonding gene reaction rule
        expression: dictionary of gene names (keys) to values from [likely] observed transcriptomics data
    outputs:
        enzyme_activity: dictionary of reactions (keys) to corresponding isozyme activity from observed data (value)
    """
    
    enzyme_activity = {}
    for rxn in syn_model.reactions:
      # Initialize enzyme_activity for this reaction to 0-value
      # Note: 0-value preserved IF this reaction doesn't have any genes in its gene reaction rule
      # Obvious example: Exchange/transport reactions don't have corresponding genes in their reaction rule, so the 0-value is preserved
      enzyme_activity[rxn] = 0.0
      
      if rxn in gpr: # ensure rxn has a gene_reaction_rule defined
        for isozyme in gpr[rxn]:
          # Initialize isozyme_activity for this isozyme to infinity
          # Note: infinity-value is preserved IF this isozyme is not present in the observed transcriptomics data
          isozyme_activity = np.inf
          for gene in isozyme:

            if gene in expression: # temporary fix: ensure gene is included in observed data
              
              isozyme_activity = np.min([isozyme_activity, expression[gene]])
          enzyme_activity[rxn] += isozyme_activity
    return enzyme_activity

# Function to convert transciptomics data to enzyme activity
def convert_transcriptomics_to_enzyme_activity(transcriptomics_data: pd.DataFrame, gpr: dict[Reaction, list[list[Gene]]]):
    """Convert transcriptomics data to enzyme activity
    inputs:
        transcriptomics_data: dataframe of transcriptomics data
        gpr: dictionary of reactions (keys) to list of list of genes (values) for the correpsonding gene reaction rule
    outputs:
        enzyme_activity_df: dataframe of enzyme activity converted from transcriptomics data
    """

    # Initialize empty dataframe
    enzyme_activity_df = pd.DataFrame()

    # Loop through each strain to convert each column of transcriptomics data
    for this_strain in transcriptomics_data.columns:
        # Create dict of genes and corresponding float values using trancsciptomics data
        expression_dict = {g: transcriptomics_data.loc[g][this_strain] for g in transcriptomics_data.index}
        expr_dict_keys = [kz for kz in expression_dict.keys()]

        # Run the gene expression to enzyme activity converter for this_strain
        enzyme_activity_dict = gene_expression_to_enzyme_activity(gpr, expression_dict)

        # Initialize empty dataframe 
        if this_strain == transcriptomics_data.columns[0]:
            # Use enzyme_activity_dict keys as the index
            enzyme_activity_df = enzyme_activity_df.reindex(enzyme_activity_dict.keys())
            # Add reaction ID column
            enzyme_activity_df['Reaction_ID'] = [k.id for k in enzyme_activity_dict.keys()]
        
        # Add enzymze_activity to dataframe
        enzyme_activity_df[this_strain] = enzyme_activity_dict
    
    return enzyme_activity_df


In [12]:
# Run enzyme activity converter for all strains in transcriptomics_df
all_enzyme_activity_df = convert_transcriptomics_to_enzyme_activity(transcriptomics_df, syn_model)


In [13]:
# Write dataframe to csv
all_enzyme_activity_df.to_csv('processed_data/enzymze_activity.csv')

all_enzyme_activity_df.iloc[0:25]

,Reaction_ID,Se_axen_d4_1,Se_axen_d4_2,Se_axen_d4_3,Se_axen_d6_1,Se_axen_d6_2,Se_axen_d6_3,Se_axen_d8_1,Se_axen_d8_2,Se_axen_d8_3
EX_gln__L_e: gln__L_e -->,EX_gln__L_e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EX_hco3_e: hco3_e <=>,EX_hco3_e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EX_mn2_e: mn2_e <=>,EX_mn2_e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EX_arg__L_e: arg__L_e -->,EX_arg__L_e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ADPT: ade_c + prpp_c <=> amp_c + ppi_c,ADPT,inf,inf,inf,inf,inf,inf,inf,inf,inf
O2tcx: o2_c --> o2_cx,O2tcx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AOXPBDC: 2a3pp_c + h_c --> 3a2oxpp_c + co2_c,AOXPBDC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DNTPPA: ahdt_c + h2o_c --> dhpmp_c + h_c + ppi_c,DNTPPA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CBMD: cbm_c + 2.0 h_c --> co2_c + nh4_c,CBMD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BIOMASS_PIGMENTS: 0.1707 calxan_c + 0.1138 caro_c + 0.5691 cholphya_c + 0.5691 hpdcn_c + 0.0569 nstxan_c + 0.0524 phycy_c + 0.2276 zeax_c --> bm_pigm_c,BIOMASS_PIGMENTS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


 ### 3. For each condition, normalize enzyme data with respect to the reference strain

In [15]:
# Create dataframe of normalized enzyme activity by scaling realtive to reference strain
enzymze_activity_df = all_enzyme_activity_df.copy()

rep_names = transcriptomics_df.columns
normalized_enzyme_activity_df = enzymze_activity_df[rep_names].div(enzymze_activity_df[ref_rep], axis=0)


# TODO: should these replacements be the original values (0 and inf), or 1s (to indicate same as reference strain)?
# ==> NEITHER: drop rows with all 0 or inf or NaN entries

# Ensure 0-valued or inf entries in enzymze_activity_df are passed into normalized_enzyme_activity_df
# - OR - 
# Ensure 0-valued or inf entries in enzymze_activity_df are passed into normalized_enzyme_activity_df
for col in rep_names:
    for row in enzymze_activity_df.index:
        if (enzymze_activity_df[col][row] == 0.0) or (np.isinf(enzymze_activity_df[col][row])):
            # this_row = enzymze_activity_df['Reaction_ID'][row]
            # normalized_enzyme_activity_df.loc[this_row, col] = enzymze_activity_df[col][row]
            normalized_enzyme_activity_df.loc[row, col] = enzymze_activity_df[col][row]

# Re-index normalized_enzyme_activity_df using reaction-IDs
normalized_enzyme_activity_df['Reaction_ID'] = enzymze_activity_df['Reaction_ID']
normalized_enzyme_activity_df = normalized_enzyme_activity_df.set_index('Reaction_ID')

normalized_enzyme_activity_df.iloc[0:25]

# TODO: add "check_data" function to check validity before passing to pymc

,Se_axen_d4_1,Se_axen_d4_2,Se_axen_d4_3,Se_axen_d6_1,Se_axen_d6_2,Se_axen_d6_3,Se_axen_d8_1,Se_axen_d8_2,Se_axen_d8_3
Reaction_ID,,,,,,,,,
EX_gln__L_e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
EX_hco3_e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
EX_mn2_e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
EX_arg__L_e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
ADPT,inf,inf,inf,inf,inf,inf,inf,inf,inf
O2tcx,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
AOXPBDC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
DNTPPA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
CBMD,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [74]:
missing_rxns = {r: (r in FVA_object.index) for r in normalized_enzyme_activity_df.index}
{k:v for k,v in missing_rxns.items() if not v}

{'BIOMASS__1': False, 'EX_co2_e': False, 'EX_sucr_e': False}

# ----- STOP HERE ..... for now

### 4. For each condition, multiply the reference bounds by the normalized enzyme data.

In [98]:
# Get relative scaled rates of change in metabolic abundance
normalized_rates_df = rates_df.div(rates_df[ref_rep], axis=0)

In [89]:
# TODO: make this into a function
flux_bounds = {}
for this_strain in normalized_enzyme_activity_df.columns:
    flux_bounds[this_strain] = FVA_object
    for this_rxn in normalized_enzyme_activity_df.index:
        if this_rxn in FVA_object:
            this_factor = normalized_enzyme_activity_df[this_strain][this_rxn]
            flux_bounds[this_strain].loc[this_rxn] = this_factor*FVA_object[this_strain][this_rxn]
        elif this_rxn in missing_rxns:
            # TODO: for sucrose production ('EX_sucr_e'), use normalized calcualted rate from normalized_rates_df
            if rxn=='EX_sucr_e':
                # flux_bounds[this_strain].loc[this_rxn] = pd.Series({'minimum': ....,
                #                                                     'maximum': ....})
                

test_df = flux_bounds['Se_axen_d4_1'].iloc[0:35]
test_df.columns = ['d4_1_min', 'd4_1_max']
test_df.join(flux_bounds[ref_rep].iloc[0:35])

,d4_1_min,d4_1_max,minimum,maximum
EX_gln__L_e,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
EX_hco3_e,-1.990000e+00,-1.990000e+00,-1.990000e+00,-1.990000e+00
EX_mn2_e,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
EX_arg__L_e,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
ADPT,1.032064e-05,1.032064e-05,1.032064e-05,1.032064e-05
O2tcx,2.631984e-02,2.631984e-02,2.631984e-02,2.631984e-02
AOXPBDC,1.411151e-06,1.411151e-06,1.411151e-06,1.411151e-06
DNTPPA,4.111581e-06,4.111581e-06,4.111581e-06,4.111581e-06
CBMD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
BIOMASS_PIGMENTS,1.061867e-03,1.061867e-03,1.061867e-03,1.061867e-03


In [82]:
FVA_object.iloc[0]

minimum    0.0
maximum    0.0
Name: EX_gln__L_e, dtype: float64

### 5. Run FBA using the condition-specific bounds to compute the condition-specific fluxes